In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from datetime import datetime
from pandas import ExcelWriter

# Configuration
RISK_THRESHOLDS = {
    'product_return': 0.3,
    'category_return': 0.25,
    'high_value': 500,
    'fast_return_days': 7
}

# 1. Data Loading and Initial Preparation
def load_and_prepare_data(filepath):
    df = pd.read_csv(filepath)
    
    # Data Cleaning
    df['Return_Reason'] = df['Return_Reason'].fillna('Not Returned')
    df['Days_to_Return'] = df['Days_to_Return'].fillna(0)

    
    # Date Handling
    df['Order_Date'] = pd.to_datetime(df['Order_Date'], dayfirst=True)
    df['Return_Date'] = pd.to_datetime(df['Return_Date'], dayfirst=True, errors='coerce')
    
    # Data Validation
    df = df[(df['Product_Price'] > 0) & (df['Order_Quantity'] > 0)]
    
    return df

# 2. Feature Engineering
def create_features(df):
    # Basic Features
    df['order_value'] = df['Product_Price'] * df['Order_Quantity']
    df['discount_ratio'] = df['Discount_Applied'] / df['Product_Price']
    
    # Time Features
    df['order_day_of_week'] = df['Order_Date'].dt.dayofweek
    df['order_month'] = df['Order_Date'].dt.month
    df['order_year'] = df['Order_Date'].dt.year
    
    # Customer Behavior
    df['customer_return_rate'] = df.groupby('User_ID')['Return_Status'].transform(
        lambda x: (x == 'Returned').mean()
    )
    df['days_since_last_order'] = df.groupby('User_ID')['Order_Date'].diff().dt.days
    
    # Product Risk Features
    product_stats = df.groupby('Product_ID').agg(
        total_orders=('Order_ID', 'count'),
        returns=('Return_Status', lambda x: (x == 'Returned').sum())
    )
    product_stats['product_return_rate'] = product_stats['returns'] / product_stats['total_orders']
    
    category_stats = df.groupby('Product_Category').agg(
        total_orders=('Order_ID', 'count'),
        returns=('Return_Status', lambda x: (x == 'Returned').sum())
    )
    category_stats['category_return_rate'] = category_stats['returns'] / category_stats['total_orders']
    
    df = df.merge(product_stats['product_return_rate'], on='Product_ID', how='left')
    df = df.merge(category_stats['category_return_rate'], on='Product_Category', how='left')
    
    # Rolling return rates
    df = df.sort_values(['Product_ID', 'Order_Date'])
    df['rolling_return_rate'] = df.groupby('Product_ID')['Return_Status'].transform(
        lambda x: (x == 'Returned').expanding().mean()
    )
    
    return df

# 3. Risk Modeling
def calculate_risk_flags(df, thresholds):
    # Basic Risk Flags
    df['high_risk_product'] = df['product_return_rate'] >= thresholds['product_return']
    df['high_risk_category'] = df['category_return_rate'] >= thresholds['category_return']
    df['high_value_order'] = df['order_value'] >= thresholds['high_value']
    
    # Behavioral Risk Flags
    df['fast_return'] = (df['Days_to_Return'] <= thresholds['fast_return_days']) & (df['Return_Status'] == 'Returned')
    df['defective_return'] = (df['Return_Reason'] == 'Defective') & (df['Return_Status'] == 'Returned')
    df['seasonal_risk'] = df['order_month'].isin([11, 12])  # Holiday season
    
    # Operational Risk Flags
    shipping_risk = {
        'Next-Day': 0.1,
        'Express': 0.15,
        'Standard': 0.25
    }
    df['shipping_risk_score'] = df['Shipping_Method'].map(shipping_risk)
    
    payment_risk = {
        'Credit Card': 0.1,
        'Debit Card': 0.15,
        'PayPal': 0.08,
        'Gift Card': 0.25
    }
    df['payment_risk_score'] = df['Payment_Method'].map(payment_risk)
    
    # Composite Risk Score
    risk_factors = [
        df['high_risk_product'].astype(int),
        df['high_risk_category'].astype(int),
        df['high_value_order'].astype(int) * 0.5,
        df['fast_return'].astype(int),
        df['defective_return'].astype(int),
        df['shipping_risk_score'],
        df['payment_risk_score']
    ]
    
    df['composite_risk_score'] = sum(risk_factors) / (len(risk_factors) - 2)  # Normalize
    df['critical_risk'] = df['composite_risk_score'] > 0.5
    
    return df

# 4. Advanced Analytics
def perform_advanced_analysis(df):
    # Product Clustering
    product_features = df.groupby('Product_ID').agg({
        'Product_Price': 'mean',
        'product_return_rate': 'mean',
        'order_value': 'mean'
    }).reset_index()
    
    kmeans = KMeans(n_clusters=5, random_state=42)
    product_features['product_cluster'] = kmeans.fit_predict(
        product_features[['Product_Price', 'product_return_rate', 'order_value']]
    )
    
    df = df.merge(product_features[['Product_ID', 'product_cluster']], on='Product_ID', how='left')
    
    # Customer Segmentation
    customer_features = df.groupby('User_ID').agg({
        'order_value': ['count', 'sum', 'mean'],
        'customer_return_rate': 'mean',
        'days_since_last_order': 'mean'
    }).reset_index()
    
    customer_features.columns = ['User_ID', 'order_count', 'total_spend', 'avg_order_value', 
                               'return_rate', 'avg_days_between_orders']
    
    kmeans = KMeans(n_clusters=4, random_state=42)
    customer_features['customer_segment'] = kmeans.fit_predict(
        customer_features[['order_count', 'total_spend', 'return_rate']]
    )
    
    df = df.merge(customer_features[['User_ID', 'customer_segment']], on='User_ID', how='left')
    
    return df

# 5. Visualization
def create_visualizations(df):
    plt.figure(figsize=(15, 10))
    
    # Return Reasons
    plt.subplot(2, 2, 1)
    sns.countplot(y='Return_Reason', 
                 data=df[df['Return_Status'] == 'Returned'],
                 order=df[df['Return_Status'] == 'Returned']['Return_Reason'].value_counts().index)
    plt.title('Primary Reasons for Returns')
    
    # Risk Score Distribution
    plt.subplot(2, 2, 2)
    sns.histplot(df['composite_risk_score'], bins=20, kde=True)
    plt.title('Distribution of Composite Risk Scores')
    
    # Risk by Category
    plt.subplot(2, 2, 3)
    risk_by_category = df.groupby('Product_Category')['composite_risk_score'].mean().sort_values()
    sns.barplot(x=risk_by_category.values, y=risk_by_category.index)
    plt.title('Average Risk Score by Product Category')
    
    # Payment Method Risk
    plt.subplot(2, 2, 4)
    sns.boxplot(x='Payment_Method', y='composite_risk_score', data=df)
    plt.title('Risk Distribution by Payment Method')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.savefig('ecommerce_risk_analysis.png')
    plt.close()
# 6. Reporting - Corrected Version
def generate_reports(df):
    # Daily Risk Report
    daily_risk = df.groupby(df['Order_Date'].dt.date).agg({
        'Order_ID': 'count',
        'critical_risk': 'sum',
        'order_value': 'sum'
    }).rename(columns={
        'Order_ID': 'total_orders',
        'critical_risk': 'high_risk_orders',
        'order_value': 'total_sales'
    })
    daily_risk['risk_percentage'] = daily_risk['high_risk_orders'] / daily_risk['total_orders']
    
    # Product Risk Report
    product_risk = df.groupby(['Product_ID', 'Product_Category']).agg({
        'Order_ID': 'count',
        'Return_Status': lambda x: (x == 'Returned').sum(),
        'order_value': 'sum',
        'composite_risk_score': 'mean'
    }).rename(columns={
        'Order_ID': 'total_orders',
        'Return_Status': 'returns',
        'order_value': 'total_sales'
    })
    product_risk['return_rate'] = product_risk['returns'] / product_risk['total_orders']
    
    # Save to Excel with formatting using xlsxwriter
    with ExcelWriter('ecommerce_risk_reports.xlsx', engine='xlsxwriter') as writer:
        df.to_excel(writer, sheet_name='All Orders', index=False)
        daily_risk.to_excel(writer, sheet_name='Daily Risk', index=True)
        product_risk.to_excel(writer, sheet_name='Product Risk', index=True)
        
        # Get workbook and worksheet for formatting
        workbook = writer.book
        worksheet = writer.sheets['All Orders']
        
        # Add conditional formatting for critical risks
        red_format = workbook.add_format({'bg_color': '#FFC7CE'})
        
        # Find the column index of composite_risk_score
        risk_col_index = df.columns.get_loc('composite_risk_score')
        risk_col_letter = chr(65 + risk_col_index)  # Convert to Excel column letter
        
        # Apply formatting to all rows in the risk column
        worksheet.conditional_format(1, risk_col_index, len(df), risk_col_index, {
            'type': 'cell',
            'criteria': '>',
            'value': 0.5,
            'format': red_format
        })

        # Auto-adjust column widths
        for sheet in writer.sheets.values():
            sheet.set_column(0, len(df.columns) - 1, 20)

# Main Execution
def main():
    print("Starting ecommerce risk analysis...")
    
    # 1. Load and prepare data
    print("Loading and preparing data...")
    df = load_and_prepare_data(r'C:\Users\TEJASWINI\Desktop\ecommerce.csv')
    
    # 2. Feature engineering
    print("Creating advanced features...")
    df = create_features(df)
    
    # 3. Risk modeling
    print("Calculating risk flags...")
    df = calculate_risk_flags(df, RISK_THRESHOLDS)
    
    # 4. Advanced analytics
    print("Performing advanced analytics...")
    df = perform_advanced_analysis(df)
    
    # 5. Visualization
    print("Creating visualizations...")
    create_visualizations(df)
    
    # 6. Reporting
    print("Generating reports...")
    generate_reports(df)
    
    # Save final dataset
    df.to_csv('enhanced_ecommerce_data.csv', index=False)
    
    print("Analysis complete!")
    print(f"Total orders analyzed: {len(df)}")
    print(f"Critical risk orders identified: {df['critical_risk'].sum()}")
    print(f"Output files created:")
    print("- enhanced_ecommerce_data.csv")
    print("- ecommerce_risk_analysis.png")
    print("- ecommerce_risk_reports.xlsx")

if __name__ == "__main__":
    main()

Starting ecommerce risk analysis...
Loading and preparing data...
Creating advanced features...
Calculating risk flags...
Performing advanced analytics...
Creating visualizations...
Generating reports...
Analysis complete!
Total orders analyzed: 10000
Critical risk orders identified: 4249
Output files created:
- enhanced_ecommerce_data.csv
- ecommerce_risk_analysis.png
- ecommerce_risk_reports.xlsx


In [4]:
import pandas as pd

# Load the data
df = load_and_prepare_data(r'C:\Users\TEJASWINI\Desktop\ecommerce.csv')
product_stats = df.groupby('Product_ID').agg(
    total_orders=('Order_ID', 'count'),
    returns=('Return_Status', lambda x: (x == 'Returned').sum())
)
product_stats['return_rate'] = product_stats['returns'] / product_stats['total_orders']
avg_days_to_return = df.groupby('Product_ID')['Days_to_Return'].mean()
product_stats = product_stats.join(avg_days_to_return.rename('avg_days_to_return'))
avg_discount = df.groupby('Product_ID')['Discount_Applied'].mean()
product_stats = product_stats.join(avg_discount.rename('avg_discount'))
product_info = df[['Product_ID', 'Product_Category', 'Product_Price']].drop_duplicates()
product_stats = product_stats.join(product_info.set_index('Product_ID'))
high_risk_threshold = 0.3
product_stats['high_risk'] = product_stats['return_rate'] >= high_risk_threshold
high_risk_products = product_stats[product_stats['high_risk']].sort_values('return_rate', ascending=False)
print(f"Found {len(high_risk_products)} high-risk products (return rate ≥ {high_risk_threshold*100}%)")

Found 5052 high-risk products (return rate ≥ 30.0%)
